In [0]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from sklearn.utils import resample,shuffle
import csv
import random
import math
import operator

In [0]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][classcol] == predictions[x]:
            correct += 1
    print(len(testSet))
    return (correct/float(len(testSet))) * 100.0
def tfpn(testSet, predictions):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][classcol] == predictions[x]:
            if(testSet[x][classcol] == 1):
                tp+=1
            else:
                tn+=1
        else:
            if(predictions[x] ==1):
                fp+=1
            else:
                fn+=1
    return (tp,tn,fp,fn)
def findclass(df):
    classs=None
    columns=df.columns.tolist()
    for i in range(len(columns)):
        if columns[i]=='class':
            classs=i
            break
    return classs
classcol=None
def upsample(data):
    df_maj = data[data['class'] == 1]
    l = len(df_maj)
    #print(len(df_maj))
    df_min = data[data['class'] == 0]
    df_minor_up = resample(df_min,replace=True,n_samples=l,random_state=123)
    #print(len(df_minor_up))
    df_upsampled = pd.concat([df_maj,df_minor_up])
    df_upsampled=shuffle(df_upsampled)
    df=df_upsampled.drop(['spectrometric_redshift','Unnamed: 0', 'galex_objid', 'sdss_objid','pred'],1)
    return df
def convert(dataset):
    dataset=list(dataset.values.tolist())
    length=len(dataset[0])
    for x in range(0,len(dataset)-1):
        for y in range(1,length):
            dataset[x][y]=float(dataset[x][y])
    return dataset
def loaddataset(file1,file2,trainset=[],testset=[]):
    data1=pd.read_csv(file1)
    data2=pd.read_csv(file2)
    l=np.intersect1d(data2.columns, data1.columns)
    data1=data1[l]
    data2=data2[l]
    data1=upsample(data1)
    data2=upsample(data2)
    global classcol
    classcol=findclass(data2)
    data1=data1[1:]
    data2=data2[1:]
    #print(len(data1))
    #print(len(data2))
    train=convert(data1)
    test=convert(data2)
    for i in range(len(train)):
        trainset.append(train[i])
    for i in range(len(test)):
        testset.append(test[i])


def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow(float(instance1[x]) - float(instance2[x]), 2)
    return math.sqrt(distance)

def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)
    for x in range(1,len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][classcol]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
        sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][classcol] == predictions[x]:
            correct += 1
    print(len(testSet))
    return (correct/float(len(testSet))) * 100.0
def main(k):
# prepare data
    trainingSet=[]
    testSet=[]
    loaddataset('/content/cat3.csv','/content/cat2.csv',trainingSet,testSet)
    # generate predictions
    predictions=[]
    for x in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        #print('predicted=' + str(result) + ', actual=' + str(testSet[x][classcol]))
    accuracy = getAccuracy(testSet, predictions)
    print(accuracy)
    l = tfpn(testSet,predictions)
    TP = l[0]
    TN = l[1]
    FP = l[2]
    FN = l[3]

    print("true positive :",TP)
    print("true negative :",TN)
    print("false positive :",FP)
    print("false negative :",FN)
    TPR = TP/(TP+FN)
    print( 'Sensitivity:{}'.format(TPR))
    TNR = TN/(TN+FP)
    print( 'Specificity:{}'.format(TNR))
    Precision = TP/(TP+FP)
    print( 'Precision:{}'.format(Precision))
    Recall = TP/(TP+FN)
    print( 'Recall:{}'.format(Recall))
    Acc = (TP+TN)/(TP+TN+FP+FN)
    print( 'Accuracy:{}'.format(Acc))
    Fscore = 2*(Precision*Recall)/(Precision+Recall)
    print( 'FScore:{}'.format(Fscore))
main(3)
  

6457
99.6747715657426
true positive : 3208
true negative : 3228
false positive : 0
false negative : 21
Sensitivity:0.9934964385258594
Specificity:1.0
Precision:1.0
Recall:0.9934964385258594
Accuracy:0.996747715657426
FScore:0.9967376106882088


In [0]:
main(5)

6457
99.31856899488926
true positive : 3187
true negative : 3226
false positive : 2
false negative : 42
Sensitivity:0.9869928770517188
Specificity:0.9993804213135068
Precision:0.9993728441517717
Recall:0.9869928770517188
Accuracy:0.9931856899488927
FScore:0.9931442817076972
